# Data Analysis and Visualization in Python
## Data Formatting
Questions
* What are the different data types in Pandas?
* What impacts have data types on descriptive statistics?
* How can I manage undefined (null) values?
* How can I save a dataframe to a file?

Objectives
* Manipulate the data types.
* Create a copy of a DataFrame.
* Transform or remove null values.
* Write modified data to a CSV file.

## Loading our data

In [ ]:
# First make sure pandas is loaded
import pandas as pd

# Read in the survey csv
surveys_df = pd.read_csv("../data/surveys.csv")

## Types of Data
### Checking the format of our data

Native Python Type | Pandas Type | Description
:-----------------:|:-----------:|:-----------
`str`              | `object`    | The most general dtype. Will be assigned to your column if column has mixed types (numbers and strings).
`int`              | `int64`     | 64 bits integer
`float`            | `float64`   | Numeric characters with decimals. If a column contains numbers and NaNs(see below), pandas will default to float64.
 N/A               | `datetime64`| Values meant to hold time data.

In [ ]:
# Getting the data type of species identifiers
surveys_df['species_id'].dtype

In [ ]:
# Getting the data type of month values
surveys_df['month'].dtype

In [ ]:
# Getting the data types of all columns
surveys_df.dtypes

### Working With Our Survey Data

In [ ]:
# Summary of descriptive statistics
surveys_df.describe()

In [ ]:
# Convert month numbers to nominal values
surveys_df['month'] = surveys_df['month'].astype('str')
surveys_df['month'].dtype

In [ ]:
# Descriptive statistics on a categorical variable
surveys_df['month'].describe()

In [ ]:
# Listing all different months
surveys_df['month'].unique()

In [ ]:
# Listing all different years
surveys_df['year'].unique()

In [ ]:
# What happens if we try to convert weight values to int64 integers?
try:
    surveys_df['weight'].astype('int64')
except BaseException as error:
    print(f'The problem: {error}')

## Selecting and cleaning undefined values

In [ ]:
# For each value, is the value undefined
surveys_df.isna()

In [ ]:
# Select rows with at least one undefined value
nan_mask = surveys_df.isna().any(axis='columns')
surveys_df[nan_mask]

In [ ]:
# What does this do?
one_selection = surveys_df[surveys_df['weight'].isna()]
one_selection.groupby('species_id')['record_id'].count()

### Getting Rid of the NaN’s

In [ ]:
def state_by_species(df, column):
    print(
        df.groupby('species_id')[column].aggregate(
            ['count', 'mean', 'std']
        ).loc['DM':'NL']
    )
    print()
    print('Total count:', df[column].count())

# Before the cleanup
state_by_species(surveys_df, 'weight')

In [ ]:
# Create a copy to avoid modifying the original object
copy_surveys = surveys_df.copy()
copy_surveys.head()

In [ ]:
# For a stable mean value per species
copy_surveys.groupby('species_id')['weight'].transform('mean')

In [ ]:
# Replace unknown values by known mean values
copy_surveys['weight'] = copy_surveys['weight'].fillna(
    copy_surveys.groupby('species_id')['weight'].transform('mean')
)

In [ ]:
# Before and after the cleanup
state_by_species(surveys_df, 'weight')
print()  # Print an empty line
state_by_species(copy_surveys, 'weight')

### Exercise - Data Cleanup
Repeat the same steps to fill in the undefined
values, but for the `'hindfoot_length'` column.
However, this time we want to calculate the averages
according to `'species_id'` and `'sex'`.

The `state_by_species_and_sex()` function is provided
to display statistics before and after cleaning.

(5 min.)

In [ ]:
def state_by_species_and_sex(df, column):
    print(
        df.groupby(
            ['species_id', 'sex']
        )[column].aggregate(
            ['count', 'mean', 'std']
        ).unstack().head()
    )
    print()
    print('Total count:', df[column].count())

In [ ]:
column = 'hindfoot_length'
state_by_species_and_sex(copy_surveys, column)
print()  # Print an empty line

copy_surveys[column] = copy_surveys[column].fillna(
    copy_surveys.groupby(
        ['species_id', 'sex']
    )[column].transform('mean')
)

state_by_species_and_sex(copy_surveys, column)

### Writing Out Data to CSV

In [ ]:
# Only keep (complete) records that have no NA
df_no_na = copy_surveys.dropna()
df_no_na

In [ ]:
# Save the cleaned DataFrame to a CSV file
df_no_na.to_csv('surveys_complete.csv', index=False)

## Technical Summary
* **Managing data types**
    * For a **DataFrame**:
        * Attribute: `dtypes`
    * For a **Series** (column):
        * Attribute: `dtype`
        * Method: `astype()`
* **Cleaning data**
    * `df.copy()`
    * `isna()`, `notna()`
    * `column.fillna(value, inplace=True)`
* **Saving a DataFrame**
    * `df.to_csv(csv_filename, index=False)`